# CDC

In [ ]:
import pandas as pd
import os

parent_root = os.path.dirname(os.getcwd())

data = pd.read_csv(f'{parent_root}/Data/Behavioral_Risk_Factor_Surveillance_System__BRFSS__-__National_Cardiovascular_Disease_Surveillance_Data.csv')

In [ ]:
drop_list = ['row_id',
             'DataSource',
             'PriorityArea1',
             'PriorityArea2',
             'PriorityArea3',
             'PriorityArea4',
             'CategoryId',
             'IndicatorID',
             'Data_Value_TypeID',
             'BreakOutCategoryId',
             'Data_Value_Unit',
             'LowConfidenceLimit',
             'HighConfidenceLimit',
             'BreakOutId',
             'LocationID',
             'TopicId',
             'Data_Value_Footnote',
             'Data_Value_Footnote_Symbol',
             'Geolocation']


data_dropped = data.drop(drop_list,axis=1)

In [ ]:
# Remove rows where Data_Value_Alt = -2

data_dropped[data_dropped['Data_Value_Alt'] != -2]

# Filter Year of 2018

data_dropped[data_dropped['Year'] == 2018]

# Filter by Category

cardio = data_dropped[data_dropped['Category'] == "Cardiovascular Diseases"]
riskfactor = data_dropped[data_dropped['Category'] == "Risk Factors"]

In [ ]:
cardio.Topic.value_counts()

In [ ]:
23437/126256

# Census

In [ ]:
import requests

var = requests.get("https://api.census.gov/data/2018/acs/acs1/subject/variables.json").json()

In [ ]:
key = "16effcabd5d51ae74821b3639db3844d8c7f1beb"

table = requests.get(f"https://api.census.gov/data/2018/acs/acs1/subject?get=group(S2701)&for=state:*&key={key}").json()

# Get indices of unwanted columns

In [ ]:
index_list = [table[0].index(x) if x[-1] == "M" or x[-2:] == "MA" or x[-2:] == "EA" else None for x in table[0]]
index_list = list(filter(lambda x: x!=None, index_list))

### Remove the columns

In [ ]:
new_table = []
for rows in table:
    
    rows = ['None' if x is None else x for x in rows]
    
    offset = 0
    for ind in index_list:
        rows.pop(ind-offset)
        offset += 1
    new_table.append(rows)

### Replace header using var json

In [ ]:
for x in new_table[0]:
    
    try:
        col = var['variables'][x]['label'].split("!!")
        ind = new_table[0].index(x)
        
        print(col)
        if len(col) > 3: 
            new_table[0][ind] = col[1] + "!!"+ "!!".join(col[3:])
        else:
            new_table[0][ind] = col[1] + "!!"+ col[-1]
    except:
        None
        

### Write to csv

In [ ]:
import csv

with open('test.csv', 'w+', newline='') as csvfile:
    spamwriter = csv.writer(csvfile,delimiter = ",")
    for rows in new_table:
        spamwriter.writerow(rows)

# Read in CSV to Dataframe & Transform Data

In [ ]:
import pandas as pd

data = pd.read_csv('test.csv')

In [ ]:
def filterColumns(dataframe,key=[]):
    columns = []
    for x in key:
        columns.append(dataframe.filter(like=x))
    return pd.concat(columns,axis=1)

# Get Table Names

In [ ]:
colname = filterColumns(data,['NAME','Total']).columns
tablename = []

for x in colname:
    if len(x.split("!!")) > 2:
        tablename.append(x.split("!!")[1])

In [ ]:
tablename = set(tablename)

In [ ]:
listofdataframes = []
for name in tablename:
    print(name)
    listofdataframes.append(filterColumns(data,['NAME',name]))

# Establish connection with sql database

In [163]:
import pymssql

#SQL config
server = "gen10-data-fundamentals-21-11-sql-server.database.windows.net"
database = "group2"
user = "group2"
password = "capstonedev10!!"
table = "heart_disease"

conn = pymssql.connect(server=server, user=user, password=password,database=database)
c = conn.cursor()

In [201]:
# convert pandas dataframe datatype to sql datatype

def dtypetosql(datatype):
    sql = []
    for x in datatype:
        #print(x)
        if x == "object":
            s = "varchar(500)"
        elif x == "int64":
            s = "int"
        elif x == "float64":
            s = "float"
        sql.append(s)
    return sql
        

### Replace whistespace with underscore

In [215]:
for i in range(len(tablename)):
    table = list(tablename)[i].replace(" ","_").replace("(","").replace(")","").replace("-","_").replace(".","_")
    sql_colname = list(map(lambda x: x.replace(" ","_").replace("!!","_").replace(table,"").replace("(","").replace(")","").replace("'","").replace(".","")\
                           .replace("Civilian_noninstitutionalized_population","CNP").replace("-","_").replace(",","")
                           ,listofdataframes[i].columns))

    # join colname with datatype
    l = list(zip(sql_colname,dtypetosql(listofdataframes[i].dtypes)))

    # dissect tuple due to zip func
    sql_colcommands = list(map(lambda x: " ".join(x),l))

    sql_str = ", ".join(sql_colcommands)
    
    try:
        c.execute(f'CREATE TABLE {table.lower().capitalize()} ({sql_str})')
    except Exception as e:
        print('\n')
    
    try:
        listofdataframes[i].to_sql(table.lower().capitalize(), conn, if_exists='replace', index = False)
    except Exception as e:
        print(e)
    #conn.commit()



Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': 'params' arg (<class 'list'>) can be only a tuple or a dictionary.


Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': 'params' arg (<class 'list'>) can be only a tuple or a dictionary.


Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': 'params' arg (<class 'list'>) can be only a tuple or a dictionary.


Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': 'params' arg (<class 'list'>) can be only a tuple or a dictionary.


Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': 'params' arg (<class 'list'>) can be only a tuple or a dictionary.


Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': 'params' arg (<class 'list'>) can be only a tuple or a dictionary.


Execution failed on sql 'SELECT name FROM sqlite_m